### 作業
請嘗試使用 keras 來定義一個直接預測 15 個人臉關鍵點坐標的檢測網路，以及適合這個網路的 loss function


Hint: 參考前面的電腦視覺深度學習基礎

### 範例
接下來的程式碼會示範如何定義一個簡單的 CNN model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 讀取資料集以及做前處理的函數
def load_data(dirname):
    # 讀取 csv 文件
    data = pd.read_csv(dirname)
    # 過濾有缺失值的 row
    data = data.dropna()

    # 將圖片像素值讀取為 numpy array 的形態
    data['Image'] = data['Image'].apply(lambda img: np.fromstring(img, sep=' ')).values 

    # 單獨把圖像 array 抽取出來
    imgs = np.vstack(data['Image'].values)/255
    # reshape 為 96 x 96
    imgs = imgs.reshape(data.shape[0], 96, 96)
    # 轉換為 float
    imgs = imgs.astype(np.float32)
    
    # 提取坐標的部分
    points = data[data.columns[:-1]].values

    # 轉換為 float
    points = points.astype(np.float32)

    # normalize 坐標值到 [-0.5, 0.5]
    points = points/96 - 0.5
    
    return imgs, points

In [3]:
# 讀取資料
imgs_train, points_train = load_data(dirname = './D39_kaggle人臉關鍵點資料集/training.csv')
print("圖像資料:", imgs_train.shape, "\n關鍵點資料:", points_train.shape)

圖像資料: (2140, 96, 96) 
關鍵點資料: (2140, 30)


In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization

In [5]:
# 定義人臉關鍵點檢測網路
model = Sequential(name='Conv_model_for_kaggle人臉關鍵點資料集')

# 定義神經網路的輸入, hidden layer 以及輸出

model.add(Conv2D(32, (3,3), strides = 1, padding = 'same', input_shape=(96,96,1), activation='relu'))
model.add(Conv2D(64, (3,3), strides = 1, padding = 'same', activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(128, (3,3), strides = 1, padding = 'same', activation='relu'))
model.add(Conv2D(256, (3,3), strides = 1, padding = 'same', activation='relu'))
model.add(BatchNormalization(momentum=0.99, epsilon=0.001))
model.add(MaxPool2D())

model.add(Conv2D(512, (3,3), strides = 1, padding = 'same', activation='relu'))
model.add(BatchNormalization(momentum=0.99, epsilon=0.001))
model.add(MaxPool2D())
model.add(Flatten())

model.add(Dense(units=512, activation='relu')) # regularizers
model.add(Dropout(rate=0.2)) # Dropout

model.add(Dense(units=512, activation='relu')) # regularizers
model.add(Dropout(rate=0.2)) # Dropout

model.add(Dense(units=30, activation='softmax'))
# 配置 loss funtion 和 optimizer
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [6]:
print(model.summary())

Model: "Conv_model_for_kaggle人臉關鍵點資料集"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 48, 256)       295168    
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2